# Modelo em Produção

* Vamos considerar um arquivo que trás novos registros de imóveis para prever preço de aluguéis

In [1]:
import pandas as pd

In [36]:
base = pd.read_csv('producao.csv', sep=';')

base2 = base

In [37]:
base.head(3)

,bairro,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel
0,Perdizes,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,NaN
1,Bela Vista,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,NaN
2,Mooca,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN


In [38]:
regiao = pd.read_excel('bairros.xlsx')

In [39]:
base = pd.merge(base, regiao, how='left', left_on='bairro', right_on='Bairro')

In [40]:
base = base.drop(['bairro', 'Bairro'], axis=1)

In [41]:
base.head(3)

,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel,Região
0,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,NaN,Oeste
1,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,NaN,Centro
2,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,Leste


In [42]:
base = pd.concat([base, pd.get_dummies(base.Região, dtype=float)], axis=1)

In [43]:
base = base.drop('Região', axis=1)

In [44]:
base.head(3)

,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel,Centro,Leste,Norte,Oeste,Sul
0,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0
1,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0
2,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0


## Importando o algoritmo criado

In [45]:
from joblib import load

reg = load('regressor.joblib')

In [46]:
X = base.drop('aluguel', axis=1)
y = base.aluguel

In [47]:
y_pred = reg.predict(X)

* Podemos agora atribuir os valores previstos pelo modelo na nossa base de dados

In [48]:
base2['aluguel'] = y_pred

In [49]:
base2.head()

,bairro,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox,aluguel
0,Perdizes,448.0,34.0,58.0,1.0,1.0,1.0,6.0,1.0,1.0,0.0,2986.020
1,Bela Vista,1106.0,218.0,74.0,3.0,2.0,1.0,5.0,1.0,1.0,0.0,3938.950
2,Mooca,50.0,83.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1751.885
3,Santana,230.0,50.0,120.0,2.0,2.0,1.0,2.0,1.0,0.0,1.0,2755.600
4,Vila Mariana,1040.0,140.0,85.0,2.0,1.0,2.0,18.0,0.0,0.0,0.0,2522.920


In [50]:
base2.to_excel('producao_com_previsao.xlsx', index=False)